In [1]:
%cd ..

/Users/kiwifern/Development/airfoil-completion


/Users/kiwifern/Development/airfoil-completion/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Create Dataset

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
from xfoil import XFoil
from xfoil.model import Airfoil


def get_cl(coord, xf=None, angle=5):
    """
    Args:
        coord (np.array): 翼型の座標. shapeは(2,248).
        xf (XFoil, optional): XFoilのインスタンス. Defaults to None.
        angle (int, optional): 迎角. Defaults to 5.
    Returns:
        _type_: _description_
    """
    if xf is None:
        xf = XFoil()
        xf.print = False

    xf.Re = 3e6
    xf.max_iter = 100
    datax, datay = coord.reshape(2, -1)  # 念の為reshape
    xf.airfoil = Airfoil(x=datax, y=datay)
    c = xf.a(angle)
    cl = c[0]
    cl = np.round(cl, 10)
    return cl

In [3]:
naca_array = np.load("./dataset/prev-study/NACA_coords.npy")
naca_perfs = np.load("./dataset/prev-study/NACA_perfs.npy")

condition = (naca_perfs[:, 1] >= 0.5) & (naca_perfs[:, 1] <= 1.2)
naca_array_filtered = naca_array[condition]
naca_perfs = naca_perfs[condition]

In [4]:
juko_array = np.load("./dataset/prev-study/juko_array.npy")
juko_perfs = np.load("./dataset/prev-study/juko_CLCD.npy")
juko_array_reducted = juko_array[::2]
juko_perfs_reducted = juko_perfs[::2]

In [5]:
coords = np.concatenate((naca_array_filtered, juko_array_reducted), axis=0)
perfs = np.concatenate((naca_perfs, juko_perfs_reducted), axis=0)
assert coords.shape[0] == perfs.shape[0]

In [6]:
records = []
for i in tqdm(range(coords.shape[0])): 
    records.append({"coord": coords[i].reshape(2,-1), "cl": perfs[i][1]})

100%|██████████| 3767/3767 [00:00<00:00, 707977.92it/s]


In [7]:
df = pd.DataFrame(records).dropna(subset=["cl"]).reset_index(drop=True)
df

,coord,cl
0,"[[1.0, 0.9998008914, 0.9995854895, 0.999351160...",0.565521
1,"[[1.0, 0.9998011477, 0.9995860601, 0.999352042...",0.560380
2,"[[1.0, 0.999801427, 0.9995866827, 0.9993530051...",0.556866
3,"[[1.0, 0.9998017289, 0.9995873566, 0.999354048...",0.553800
4,"[[1.0, 0.9998020531, 0.9995880809, 0.99935517,...",0.550643
...,...,...
3762,"[[0.975888801765026, 0.9730030382867585, 0.969...",0.651886
3763,"[[0.9759899519714701, 0.973115741570424, 0.970...",0.644636
3764,"[[0.9760907785262438, 0.973228090364221, 0.970...",0.631992
3765,"[[0.9761912828106184, 0.9733400861306069, 0.97...",0.615084


In [8]:
# --- 配列化・標準化 ---
coords_stack = np.stack(df["coord"].values)  # shape: (N, 2, 248)
cl_array = df["cl"].values.astype(np.float32)

coord_mean = coords_stack.mean(axis=0)  # shape: (2, 248)
coord_std = coords_stack.std(axis=0)
cl_mean = cl_array.mean()
cl_std = cl_array.std()

# --- 保存（接頭辞つき）---
np.save("./dataset/NACA&Joukowski_coords_array.npy", coords_stack)
np.save("./dataset/NACA&Joukowski_cl_array.npy", cl_array)
np.savez(
    "./dataset/NACA&Joukowski_normalization_stats.npz",
    coord_mean=coord_mean,
    coord_std=coord_std,
    cl_mean=np.array([cl_mean]),
    cl_std=np.array([cl_std]),
)

In [ ]:
# cl_s = []
# for array in tqdm(naca_array):
#     cl = get_cl(array)
#     cl_s.append(cl)
# cl_s = np.array(cl_s)

# # 24min for 3696 samples

# np.save("dataset_hscvae/NACA_cls_calculated_by_xfoil_python_20250412.npy", cl_s)

100%|██████████| 3696/3696 [24:04<00:00,  2.56it/s]   


### Pytorch Dataset

In [9]:
import torch
import numpy as np


# ============================================================
# 3. データローダー (AirfoilDataset) の読み込み
# ============================================================
class AirfoilDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        coord_path="./dataset/NACA&Joukowski_coords_array.npy",
        cl_path="./dataset/NACA&Joukowski_cl_array.npy",
        norm_path="./dataset/NACA&Joukowski_normalization_stats.npz",
        normalize=True,
    ):
        coords_array = np.load(coord_path).astype(np.float32)  # shape: (N, 2, 248)
        cls_array = np.load(cl_path).astype(np.float32)[:, np.newaxis]  # shape: (N, 1)

        norm = np.load(norm_path)
        self.coord_mean = norm["coord_mean"]
        self.coord_std = norm["coord_std"]
        self.cl_mean = norm["cl_mean"][0]
        self.cl_std = norm["cl_std"][0]

        if normalize:
            coords_array = (coords_array - self.coord_mean) / self.coord_std
            cls_array = (cls_array - self.cl_mean) / self.cl_std

        self.coords_tensor = torch.tensor(coords_array, dtype=torch.float32)
        self.cls_tensor = torch.tensor(cls_array, dtype=torch.float32)
        self.normalize = normalize

    def __len__(self):
        return self.coords_tensor.shape[0]

    def __getitem__(self, idx):
        return self.coords_tensor[idx], self.cls_tensor[idx]

    def denormalize_coord(self, coord_tensor):
        std = torch.tensor(self.coord_std, dtype=torch.float32, device=coord_tensor.device)
        mean = torch.tensor(self.coord_mean, dtype=torch.float32, device=coord_tensor.device)
        return coord_tensor * std + mean

    def normalize_cl(self, cl_tensor):
        std = torch.tensor(self.cl_std, dtype=torch.float32, device=cl_tensor.device)
        mean = torch.tensor(self.cl_mean, dtype=torch.float32, device=cl_tensor.device)
        return (cl_tensor - mean) / std

    def denormalize_cl(self, cl_tensor):
        std = torch.tensor(self.cl_std, dtype=torch.float32, device=cl_tensor.device)
        mean = torch.tensor(self.cl_mean, dtype=torch.float32, device=cl_tensor.device)
        return cl_tensor * std + mean

In [10]:
from torch.utils.data import DataLoader

dataset = AirfoilDataset(normalize=True)

loader = DataLoader(dataset, batch_size=32)

for x, y in loader:
    print(x.shape)  # torch.Size([32, 2, 248])
    print(y.shape)  # torch.Size([32, 1])
    break

# 復元
x0_denorm = dataset.denormalize_coord(x[0])  # → shape (2, 248)
y0_denorm = dataset.denormalize_cl(y[0])  # → float or shape (1,)

torch.Size([32, 2, 248])
torch.Size([32, 1])
